In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb

In [2]:
def PP(nonlist):
    print(list(nonlist))
    return(nonlist)

In [3]:
#    Dictionaries and lists
important=['Participant ID','Date of Intake',
     'Whole Grains (ounce equivalents)','Sodium (mg)','Refined Grains (ounce equivalents)',
     'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
     'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)',
     'FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
     'VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
     'VEG0600','VEG0900','GRW1100','GRS0100','GRR0100','GRW0500','GRS0500',
     'MSC0900','MSC1000','VEG0500','DOT0700','DOT0800','BVS0400','BVA0400',
     'BVU0300','BVS0300','BVA0300','BVS0500','BVA0500', 'BVU0400','BVS0100', 
     'BVA0100', 'BVU0100', 'BVS0200','BVA0200','BVU0200','BVS0600', 'BVA0600',
     'BVU0500', 'BVS0700', 'BVA0700', 'BVU0600','GRW0400','GRS0400','GRR0400','GRW0600',
     'GRS0600','GRR0600','GRW0700','GRW0700','GRS0700','GRR0700','GRW0900','GRS0900',
     'GRR0900','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
     'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0200',
     'MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0700','FMC0100',
     'FMC0200','MOF0500', 'MOF0600','FCF0100','SWT0700','SWT0800','GRW0200','GRS0200','GRR0200',
     'GRW0300','GRS0300','GRR0300','SWT0300','GRW0800','GRS0800','GRR0800','GRW1000','GRS1000',
     'GRR1000','SWT0100','SWT0200','SWT0400','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
     'DMR0200','DML0200',
     'DML0300','DML0400','DYF0100', 'DYR0100','DYL0100','DYR0200',
     'DYL0200','DYF0200', 'DYN0100',
     'DYN0100','DOT0300','DOT0400', 'DOT0500', 'DOT0600',
     'SWT0600','MOF0300','MOF0400','DCF0100','DCR0100','DCL0100','DCN0100','DOT0200', 'MSC0600', 'SWT0500', 'DOT0100'
     ]

#'DML0500', 'DYF0300', 'DYL0300', 'DYR0300', 'MSC1300', 'DOT0900'
units={
  'keep_serv':['SWT0100','SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400','DOT0100',
               'DOT0200','GRR0800' , 'GRS0800',  'GRW0800','MSC0900','MSC0600'],
  'cup':{
    '.5':
      ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','GRW1100','GRS0100','GRR0100','GRW0500','GRS0500',
       'MSC1000'],
    '1':
      ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
       'DML0300','DML0400','DYF0100', 'DYR0100','DYL0100','DYR0200',
       'DYL0200','DYF0200', 'DYN0100',
       'DYN0100','DOT0500', 'DOT0600',
       'SWT0600']
       },
  'floz':{
    '4':
      ['VEG0500','FRU0100','FRU0200'],
    '5':
      ['DOT0700','DOT0800'],
    '8':
      ['BVS0400','BVA0400','BVU0300','BVS0300','BVA0300','BVS0500','BVA0500', 'BVU0400',
       'BVS0100', 'BVA0100', 'BVU0100', 'BVS0200','BVA0200','BVU0200','BVS0600', 'BVA0600',
       'BVU0500', 'BVS0700', 'BVA0700', 'BVU0600']
  },
  'oz':{
    '.5':
        ['MOF0500'],
    '1':
      ['GRW0400','GRS0400','GRR0400','GRW0600',
         'GRS0600','GRR0600','GRW0700','GRW0700','GRS0700','GRR0700','GRW0900','GRS0900',
         'GRR0900','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
         'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0200',
         'MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0700','FMC0100',
         'FMC0200'],
    '2':
      ['DCF0100','DCR0100','DCL0100','DCN0100'],
    '28':
      ['GRW0200','GRS0200','GRR0200'],
    '45':
      ['GRW0300','GRS0300','GRR0300']
  },
  'TB':{
    '1':
      ['MOF0600','FCF0100']
  },
  'g':{
    '40':
      ['GRW1000','GRS1000','GRR1000']
  },
  'egg':{
    '1':
      ['MOF0300','MOF0400']
  }
}      



     
      
ped811_dict = {
    'HEI_TOTALVEG': {'parameters':[0.1,1.9], 'name': 'HEIX1_VEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[20,28,8,35], 'name': 'HEIX4_TOTALDAIRY'},
    'HEI_TOTALPROTEINS': {'parameters':[2.5,6.0,0,10.0], 'name': 'HEIX5_PROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped1224_dict = {
    'HEI_TOTALVEG': {'parameters':[0.1,7.9], 'name': 'HEIX1_VEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_TOTALDAIRY'},
    'HEI_TOTALPROTEINS': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_PROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
    }

ped08_dict = {
    'HEI_TOTALVEG': {'parameters':[0,0], 'name': 'HEIX1_VEGETABLES'},
    'HEI_TOTALFRUIT': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'HEI_WHOLEGRAINS': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'HEI_MILK': {'parameters':[0,0,0,0], 'name': 'HEIX4_TOTALDAIRY'},
    'HEI_TOTALPROTEINS': {'parameters':[0,0,0,0], 'name': 'HEIX5_PROTEIN'},
    'HEI_REFINEDGRAINS': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'HEI_FRUITJUICE': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'HEI_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'HEI_SWEETS': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'HEI_SALTY': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'HEI_CEREAL': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
    }

hei_ped_dict={
    ########### SWEETS ###########
    'HEI_SWEETS' :
        ['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good','baby food dessert','misc dessert'],
    'chocolate_candies':
        ['SWT0100'],
    'candies':
        ['SWT0200'],
    'frosting':
        ['SWT0300'],
    'sweet_sauce':
        ['SWT0700' , 'SWT0800'],
    'sugar':
        ['SWT0400'],
    'syrups':
        ['SWT0500'],
    'Pudding':
        ['DOT0300' , 'DOT0400'],
    'icecream':
        ['DOT0100' ],
    'nondairy_treat':
        ['DOT0200' ],
    'baked_good':
        ['GRR0800' , 'GRS0800',  'GRW0800' ],
    'baby food dessert':
        ['MSC0900'],
    'misc dessert':
        ['MSC0600'],
    ########### SALTY ###########
    'HEI_SALTY' :
        ['chips','other_fried','fries'],
    'chips':
        ['GRW0900' ,'GRS0900' , 'GRR0900' , 'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400'],
        'fries':
        ['VEG0800' ],
    'other_fried':
        ['FMC0100', 'FMC0200' ],
    ########### VEGGIES ###########
    'HEI_TOTALVEG':
        ['hei_veg', 'hei_greensbeans'],
    'hei_veg':
        ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
    'hei_greensbeans':
        ['VEG0100','VEG0700'],
    ########### FRUIT ###########
    'HEI_TOTALFRUIT':
        ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
    'hei_wholefruit':
        ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
    ########### GRAINS ###########
    'HEI_WHOLEGRAINS':
        ['Whole Grains (ounce equivalents)'],
    'HEI_REFINEDGRAINS':
        ['Refined Grains (ounce equivalents)'],
    ########### PROTEIN ###########
    'HEI_TOTALPROTEINS':
        ['hei_proteins','hei_seafoodplantprot','hei_dairyprot'],
    'hei_proteins':
        ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
        'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
        'MFF0100','MFL0100','MFF0200','MSL0100',
        'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
        'MOF0600','MOF0700','VEG0700'],
    'hei_seafoodplantprot':
        ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
    ########### DAIRY ###########
    'HEI_MILK':
        ['milk','formula_foz'],
    'hei_dairy':
        ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
        'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
    'hei_dairyprot':
        ['DCF0100','DCR0100','DCL0100','DCN0100','DCN0100','DYF0100',
        'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100'],
    'milk':
        ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
    'formula_foz':
        ['DOT0700', 'DOT0800'],
    ########### CEREAL ###########
    'HEI_CEREAL':
        ['cereal_oz', 'bbcereal_hcup'],
    'cereal_oz':
        ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'bbcereal_hcup':
        ['GRW1300','GRS1300'],
    ########### JUICES ###########
    'HEI_FRUITJUICE':
        ['FRU0100','FRU0200'],
    ########### SSB ###########
    'HEI_SSB':
        ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
    ########### CALORIES ###########
    'ENERGY':
        ['Energy (kcal)'] 
        }

conv_dict={
    'hei_fruitjuice':4,
    'hei_SSB':8,
    'chocolate_candies':40,
    'candies':15,
    'frosting':35,
    'sugar':4,
    'syrups':.25,
    'Pudding':1,
    'icecream':.5,
    'nondairy_treat':85,
    'baked_good':55,
    'fries': 70,
    'formula_foz':5,
    'bbcereal_hcup':.5
}

# make_hei_dict={
# 'hei_sweets' :
# ['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good'],
# 'hei_salty' :
# ['chips','other_fried','fries'],
# 'hei_vegetables':
# ['hei_totveg', 'hei_greensbeans'],
# 'hei_proteins':
# ['hei_totproteins','hei_seafoodplantprot'],
# 'hei_milk':
# ['hei_dairy','formula_foz'],
# 'hei_cereal':
# ['cereal_oz', 'bbcereal_hcup']
# }

ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
            'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
            'chips','other_fried','fries']

In [4]:
basepath='/Users/gracer/Google Drive/BCP/data'
arglist={'BASEPATH': basepath, 
         'CHILD': True, 
         'NAMES': 'testies', 
         'OPTS': ['Chil'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA': 'total_demo_dataset.csv'}

In [5]:
# Find the data
for (dirpath, dirnames, filenames) in os.walk(basepath):
   for filename in filenames:
       if filename.endswith('.zip'):
           tmppath=os.sep.join([dirpath, filename])
           with ZipFile(tmppath, 'r') as zipObj:
              # Get a list of all archived file names from the zip
              listOfFileNames = zipObj.namelist()
              # Iterate over the file names
              for fileName in listOfFileNames:
                  # Check filename endswith txt
                   if fileName.endswith('04.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                   if fileName.endswith('09.txt'):
                       zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')


In [6]:
def ager(date1,date2):
    # Get the age in months of each recall
    diff_time = pd.to_datetime(date1)-pd.to_datetime(date2)
    diff_time= diff_time/ np.timedelta64(1, 'M')
    return(diff_time)


In [7]:
infile

'/Users/gracer/Google Drive/BCP/data/temp_txt'

In [8]:
def file_org(infile, arglist):
    # will create a dictionary with the file paths to all the data
    if arglist['OPTS'] == False:
        arglist['OPTS']=[arglist['NAMES']]
        file_dict = {"set_04": {"%s"%arglist['NAMES']:{}}, "set_09": {"%s"%arglist['NAMES']: {}}}
        file_dict['set_04']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "%s"%arglist['NAMES'] in x]
        file_dict['set_09']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "%s"%arglist['NAMES'] in x]
        arglist['OPTS']=[arglist['NAMES']]

    else:
        file_dict={"set_04":{},"set_09":{}}
        for item in arglist['OPTS']:
            print('this is the item %s'%item)
            file_dict["set_04"][item]= {}
            file_dict["set_09"][item]= {}

        for key,value in file_dict.items():
            keys=list(file_dict[key].keys())
            for k in keys:
                print(k)
                file_dict['set_04']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if '%s'%k in x]
                file_dict['set_09']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if '%s'%k in x]

    return(file_dict)


In [9]:
a=file_org(infile, arglist) #list of the files


this is the item Chil
Chil
Chil


In [10]:
demo_interest = ["child_feeding_practice-age_any_formula", "child_feeding_practice-age_any_formula_status", "child_feeding_practice-age_fed", "child_feeding_practice-age_fed_dropdown",	"child_feeding_practice-age_fed_status", "child_feeding_practice-age_regular_formula",	"child_feeding_practice-age_regular_formula_status", "child_feeding_practice-age_stop", "child_feeding_practice-age_stop_dropdown", "child_feeding_practice-age_stop_formula", "child_feeding_practice-age_stop_formula_dropdown","child_feeding_practice-age_stop_formula_status","child_feeding_practice-age_stop_status","child_feeding_practice-any_formula",	"child_feeding_practice-birth_order",	"child_feeding_practice-breastfed",	"child_feeding_practice-four_month_ratio",	"child_feeding_practice-regular_formula", "Identifiers_visit", "Date_taken","weight","head_circumference","height","Participant ID","Gender","DoB"]


In [11]:
# proto file reader
data_dict={"set_04":{},"set_09":{},"demo":{}}
df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")

df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
df_demo['Identifiers_visit']=df_demo['Identifiers_visit'].astype('int32')
df_demo['child_feeding_practice-breastfed']=df_demo['child_feeding_practice-breastfed'].astype('category')

data_dict["demo"]= df_demo[demo_interest]
dob=data_dict['demo'][['DoB','Participant ID']].drop_duplicates(keep='last') 


In [12]:
for key, value in a.items():
    print(key)
    for k,v, in value.items():
        print(k)
        temp_list = []
        for file in v["files"]:
            print(file)
            temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
            if key == 'set_09':
                temp_df=temp_df.drop([0]) #drops extra row 
            for val in temp_df["Participant ID"]:
                _id = str(val).lstrip("0").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
        dfm_original = pd.concat(temp_list, ignore_index=True)
        print("Final dataframe size: ", dfm_original.shape)
        dfm_original = dfm_original.sort_values(by="Participant ID")
         
        del_cols=set(dfm_original.columns) - set(important)
        dfm_original.drop(del_cols, axis=1, inplace=True)
        for col in dfm_original:
            if dfm_original[col].dtype == np.object_:
                dfm_original[col] = (dfm_original[col].replace(',','.', regex=True))   
        
        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset_%s.csv'%(k,key))

        dfm_original['Participant ID'] = dfm_original['Participant ID'].astype('float').astype('int32')
        dfm_original.sort_values("Participant ID", inplace=True) 

        dfm_original=dfm_original.drop_duplicates(keep='first') 
        
       
        dfm_original=pd.merge(dob, dfm_original,on='Participant ID')
        dfm_original['age at intake']=ager(dfm_original['Date of Intake'], dfm_original['DoB'])
        
        data_dict[key]= dfm_original


set_04
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP04.txt
Final dataframe size:  (938, 226)
set_09
Chil
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1809.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1809.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild09.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly09.txt
Final dataframe size:  (938, 171)


In [13]:
data_dict['set_09'].sort_values("Participant ID", inplace=True) 

In [14]:
data_dict['set_09'][['FRU0100','FRU0200']]

,FRU0100,FRU0200
499,0,0
500,0,0
501,0,0
548,0,0
547,0,0
...,...,...
205,0,0
206,0,0
199,0,0
600,0,0


## Diet Quality Index Scoring for infants 8-24 months
| Component  | Score  | 8-11 months  | 12-24 months  |
|---|---|---|---|
| Milk  |   |   |   |
|  Breastfeeding | 10  | Yes  | Yes  |
| Formula/Cow's milk/non-dairy milks (fl oz/d) | 5  | 20-28  | 14-18  |
|   | 2.5  | 8.0–19.9 or 28.1–35.0  | 8.0–13.9 or 18.1–24.0  |
|   | 0  | <8 or >35  | <8.0 or >24.0  |
| Grains  |   |   |   |
|  Whole grains (oz/d) | 2.5  | 1.0–3.5  | 1.5–5.5  |
|   | 1.25  |  0.1–0.9 or 3.6–8.0 |  0.1–1.4 or 5.6–8.0 |
|   |  0 | 0 or >8.0  | 0 or >8.0  |
|  Refined grains (oz/d) | 2.5  |  0–1.5 |  0–1.8 |
|   |  1.25 |  1.6–3.5 |  1.9–4.2 |
|   | 0  | >3.5  | >4.2  |
| Proteins (oz/d) |  5 | 2.5–6.0  |  2.0–3.0 |
|   | 2.5  |  0.1–2.4 or 6.1–10.0 | 0.1–1.9 or 3.1–6.0  |
|   | 0  | 0 or >10  |  0 or >6.0 |
| Vegetables (oz/d) | 5  |  ≥2.0  | ≥8.0  |
|   | 2.5  |  0.1–1.9 | 0.1–7.9  |
|   |  0 |  0 |  0 |
| Fruits (oz/d) | 5  | ≥2.0  | ≥8.0  |
|   | 2.5  |  0.1–1.9 | 0.1–7.9  |
|   | 0  |  0 |  0 |
| 100% fruit juice (fl oz/d) | 5  |  0 |  0-4.0 |
|   | 2.5  | 0.1–6.0  | 4.1–6.0  |
|   |  0 | >6.0  | >6.0  |
| Sugar sweetened beverages (fl oz/d) | 5  |  0 | 0  |
|   | 2.5  | 0.1–4.0  | 4.1–6.0  |
|   |  0 | >6.0  | >6.0  |
| Sweets servings/d | 5  |  0 |  0 |
|   | 2.5  |  0.1-1.0 | 0.1-1.0  |
|   | 0  | >1.0  | >1.0  |
| Salty snacks oz/ds | 5  |  0 | 0  |
|   | 2.5  |  0.1-1.0 | 0.1-1.0  |
|   | 0  | >1.0  | >1.0  |

In [15]:
def cup2oz(cup):
    cup=cup.astype('float32')
    oz=cup*8
    return(oz)

def gram2oz(gram):
    oz=gram/28.3495
    return(oz)

def T2oz(T):
    oz=T/2
    return(oz)

def grammar(key,x,num, data):
    data[key] = gram2oz(num*(data[x].astype('float').sum(axis=1)))
    return(data[key])

def from_cup(key,x,num, data):
    data[key] = cup2oz(num*(data[x].astype('float').sum(axis=1)))
    return(data[key])

def to_fluid(key, data, num):
    data[key] = num*(data[key].astype('float'))
    return(data[key])

def to_ounce(key, x, data):
    data[key] = data[x].astype('float').sum(axis=1)
    data[key] = cup2oz(data[key]/2)
    return(data[key])

def cow_stuff(key, value, data):
    x=value[:-1]
    tmp= data[x].astype('float32').sum(axis=1)
    y=value[-1]
    if key == 'hei_dairy':
        if y == 'DOT0100':
            tmp2=data[y].astype('float32')/3
            tmp3=tmp+tmp2
            data[key]=cup2oz(tmp3)
        else:
            print('NO DAIRY MISSING DOT0100, needs to be last in list')
    if key == 'hei_totproteins':
        if y == 'VEG0700':
            tmp2=data[y].astype('float32')*2 # this is normally 1/2
            data[key]=tmp+tmp2
        else:
            print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
    if key == 'hei_seafoodplantprot':
        if y == 'VEG0700':
            tmp2=data[y].astype('float32')*2
            data[key]=tmp+tmp2
        else:
            print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
    return(data[key])

In [16]:
hei_ped_dict.keys()

dict_keys(['HEI_SWEETS', 'chocolate_candies', 'candies', 'frosting', 'sweet_sauce', 'sugar', 'syrups', 'Pudding', 'icecream', 'nondairy_treat', 'baked_good', 'baby food dessert', 'misc dessert', 'HEI_SALTY', 'chips', 'fries', 'other_fried', 'HEI_TOTALVEG', 'hei_veg', 'hei_greensbeans', 'HEI_TOTALFRUIT', 'hei_wholefruit', 'HEI_WHOLEGRAINS', 'HEI_REFINEDGRAINS', 'HEI_TOTALPROTEINS', 'hei_proteins', 'hei_seafoodplantprot', 'HEI_MILK', 'hei_dairy', 'hei_dairyprot', 'milk', 'formula_foz', 'HEI_CEREAL', 'cereal_oz', 'bbcereal_hcup', 'HEI_FRUITJUICE', 'HEI_SSB', 'ENERGY'])

In [17]:
def cup2oz(cup):
    cup=cup.astype('float32')
    oz=cup*8
    return(oz)

def gram2oz(gram):
    gram=gram.astype('float32')
    oz=gram/28.3495
    return(oz)

def T2oz(T):
    T=T.astype('float32')
    oz=T/2
    return(oz)

def egg2oz(egg):
    egg=egg.astype('float32')
    oz=egg*25.5 #about a large egg
    return(oz)

def serv2oz(serv):
    serv=serv.astype('float32')
    return(serv)

In [18]:
data_dict['set_09']['DOT0200']

499        0
500        0
501        0
548    0.612
547        0
       ...  
205        0
206        0
199        0
600        0
601        0
Name: DOT0200, Length: 604, dtype: object

In [19]:
df_conv=data_dict['set_09'][['DoB', 'Participant ID', 'Date of Intake','age at intake']+units['keep_serv']]

for key, value in units.items():
    print(key)
    if key == 'cup':
        for unit, cols in value.items():
            df_conv=df_conv.join(cup2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'TB':
        for unit, cols in value.items():
            df_conv=df_conv.join(T2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'g':
        for unit, cols in value.items():
            df_conv=df_conv.join(gram2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'egg':
        for unit, cols in value.items():
            df_conv=df_conv.join(egg2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'oz':
        for unit, cols in value.items():
            df_conv=df_conv.join(serv2oz(data_dict['set_09'][cols])*float(unit))
    if key == 'floz':
        for unit, cols in value.items():
            df_conv=df_conv.join(serv2oz(data_dict['set_09'][cols])*float(unit))

keep_serv
cup
floz
oz
TB
g
egg


In [33]:
data_dict['set_09']['DOT0200'].astype('float32').describe()

count    604.000000
mean       0.006974
std        0.064866
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.699000
Name: DOT0200, dtype: float64

In [35]:
df_conv['DOT0200'].astype('float32').describe()

count    604.000000
mean       0.006974
std        0.064866
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.699000
Name: DOT0200, dtype: float64

In [22]:
PP(df_conv.columns)

['SWT0100', 'SWT0200', 'SWT0300', 'SWT0700', 'SWT0800', 'SWT0400', 'SWT0500', 'DOT0300', 'DOT0400', 'DOT0100', 'DOT0200', 'GRR0800', 'GRS0800', 'GRW0800', 'MSC0900', 'MSC0600', 'DoB', 'Participant ID', 'Date of Intake', 'age at intake', 'FRU0300', 'FRU0400', 'FRU0500', 'FRU0600', 'FRU0700', 'VEG0100', 'VEG0200', 'VEG0300', 'VEG0400', 'VEG0800', 'VEG0450', 'VEG0700', 'VEG0600', 'VEG0900', 'GRW1100', 'GRS0100', 'GRR0100', 'GRW0500', 'GRS0500', 'MSC1000', 'DMF0100', 'DMR0100', 'DML0100', 'DMN0100', 'DMF0200', 'DMR0200', 'DML0200', 'DML0300', 'DML0400', 'DYF0100', 'DYR0100', 'DYL0100', 'DYR0200', 'DYL0200', 'DYF0200', 'DYN0100', 'DYN0100', 'DOT0500', 'DOT0600', 'SWT0600', 'VEG0500', 'FRU0100', 'FRU0200', 'DOT0700', 'DOT0800', 'BVS0400', 'BVA0400', 'BVU0300', 'BVS0300', 'BVA0300', 'BVS0500', 'BVA0500', 'BVU0400', 'BVS0100', 'BVA0100', 'BVU0100', 'BVS0200', 'BVA0200', 'BVU0200', 'BVS0600', 'BVA0600', 'BVU0500', 'BVS0700', 'BVA0700', 'BVU0600', 'MOF0500', 'GRW0400', 'GRS0400', 'GRR0400', 'GRW

Index(['SWT0100', 'SWT0200', 'SWT0300', 'SWT0700', 'SWT0800', 'SWT0400',
       'SWT0500', 'DOT0300', 'DOT0400', 'DOT0100',
       ...
       'GRW0300', 'GRS0300', 'GRR0300', 'MOF0600', 'FCF0100', 'GRW1000',
       'GRS1000', 'GRR1000', 'MOF0300', 'MOF0400'],
      dtype='object', length=140)

### dv_conv is all in oz except(!) the sweets which are in "servings" as defined by NDSR

In [41]:
for key, value in hei_ped_dict.items():
    if key == 'HEI_SWEETS':
        print(hei_ped_dict[value])
#         for food in value:
#             print(food)
#             print(hei_ped_dict[food])

TypeError: unhashable type: 'list'

In [37]:
hei_ped_dict

{'HEI_SWEETS': ['chocolate_candies',
  'candies',
  'frosting',
  'sweet_sauce',
  'sugar',
  'syrups',
  'Pudding',
  'icecream',
  'nondairy_treat',
  'baked_good',
  'baby food dessert',
  'misc dessert'],
 'chocolate_candies': ['SWT0100'],
 'candies': ['SWT0200'],
 'frosting': ['SWT0300'],
 'sweet_sauce': ['SWT0700', 'SWT0800'],
 'sugar': ['SWT0400'],
 'syrups': ['SWT0500'],
 'Pudding': ['DOT0300', 'DOT0400'],
 'icecream': ['DOT0100'],
 'nondairy_treat': ['DOT0200'],
 'baked_good': ['GRR0800', 'GRS0800', 'GRW0800'],
 'baby food dessert': ['MSC0900'],
 'misc dessert': ['MSC0600'],
 'HEI_SALTY': ['chips', 'other_fried', 'fries'],
 'chips': ['GRW0900',
  'GRS0900',
  'GRR0900',
  'GRW1100',
  'GRW1200',
  'GRW0400',
  'GRS0400',
  'GRR0400'],
 'fries': ['VEG0800'],
 'other_fried': ['FMC0100', 'FMC0200'],
 'HEI_TOTALVEG': ['hei_veg', 'hei_greensbeans'],
 'hei_veg': ['VEG0100',
  'VEG0200',
  'VEG0300',
  'VEG0400',
  'VEG0800',
  'VEG0450',
  'VEG0700',
  'VEG0600',
  'VEG0900',
  'VEG

In [23]:
# x=PP(data_dict['set_09'].columns)
# len(x)

In [24]:
# data_dict['set_09'][units['.5cup']]

In [25]:
# def make_ped_components(hei_ped_dict, complete_df, conv_dict):
#     for key, value in hei_ped_dict.items():
# #         print(value)
#         for item in value: 
#             if item in list(complete_df.columns):
#                 complete_df[item]=complete_df[item].astype('float32')
#                 if key in ['hei_frukpitjuice', 'hei_SSB','formula_foz']:
# #                     pdb.set_trace()
#                     print(complete_df[item])
#                     complete_df['%s_new'%item]=to_fluid(item, complete_df, conv_dict[key])
#                     print(complete_df['%s_new'%item])
#                 if key in ['hei_totveg','hei_greensbeans', 'hei_wholefruit','hei_totfruit']:
#                     complete_df['%s_new'%item]=to_ounce(item, value, complete_df)
#                 if key in ['cereal_oz', 'bbcereal_hcup']:
#                     if key == 'bbcereal_hcup':
#                         complete_df['%s_new'%item]=from_cup(item, value, conv_dict[key] , complete_df)
#                     else:
#                         x=value
#                         complete_df['%s_new'%item]= complete_df[x].astype('float').sum(axis=1)
#                 if key in ['hei_wholegrains','hei_refinedgrains']:
#                     complete_df['%s_new'%item] = complete_df[value].astype('float')
#                 if key in ['hei_dairy','hei_totproteins','hei_seafoodplantprot']:
#                     complete_df['%s_new'%item]=cow_stuff(item, value, complete_df)
#                 if key in ['chocolate_candies', 'candies', 'frosting', 'sugar','nondairy_treat','baked_good', 'fries']:
#                     complete_df['%s_new'%item]=grammar(item, value, conv_dict[key], complete_df)
#                 if key in ['sweet_sauce']:
#                     x=value
#                     complete_df['%s_new'%item]= T2oz(complete_df[x].astype('float').sum(axis=1))
#                 if key in ['syrups','Pudding','icecream']:
#                     from_cup(item, value, conv_dict[key], complete_df)
#                 if key in ['chips','other_fried']:
#                     x=value
#                     complete_df['%s_new'%item]= complete_df[x].astype('float').sum(axis=1)
#     return(complete_df)

In [26]:
# update_dict={'set_04':{},'set_09':{}}
# for key, value in data_dict.items():
#     update_dict[key]=make_ped_components(hei_ped_dict, value, conv_dict)

In [27]:
# data_dict['set_09']

In [28]:
# update_dict['set_09']['FRU0100_new']

In [29]:
# update_dict['set_09']['FRU0100']

In [30]:
# #e = HEI.diet_maker(b,c, arglist['NAMES'])
# df_demo=pd.read_csv(os.path.join(basepath,arglist['XTRA']), sep=",")
# # e['Participant ID'] = e['Participant ID'].astype('int32')
# # df_demo['Participant ID'] = df_demo['Participant ID'].astype('int32')
# # df = pd.merge(e,df_demo, on=['Participant ID'])
# # df['Age_at_Intake']=DQI.ager(df['Date of Intake'], df['DoB'])
# #x=DQI.BCP(df, arglist)
# #y=DQI.make_ped_components(hei_ped_dict, x, conv_dict)
# que=HEI.make_hei(y, make_hei_dict)
# split_dict=DQI.splitter(que)
# for key, value in split_dict.items():
#    if key == 'DF_child':
#        DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'child')
#        DQI.check(ped_dict['child'], value, 'child', key, arglist)
#    elif key == 'DF_young':
#        DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'young')
#        DQI.check(ped_dict['young'], value, 'young', key, arglist)
#    else:
#        DQI.DQI_BF(value, 'HEIX0_BREASTFEEDING', 'infant')
#        DQI.check(ped_dict['infant'], value, 'infant', key, arglist)
